### Data Engineering Capstone Project

#### Project Summary

##### Introduction

A core responsibility of The National Travel and Tourism Office (NTTO) is to collect, analyze, and disseminate international travel and tourism statistics. 

NTTO's Board of Managers are charged with managing, improving, and expanding the system to fully account and report the impact of travel and tourism in the United States. The analysis results help to forcecast and operation, support make decision creates a positive climate for growth in travel and tourism by reducing institutional barriers to tourism, administers joint marketing efforts, provides official travel and tourism statistics, and coordinates efforts across federal agencies.

##### Project Description

In this project, some source datas will be use to do data modeling:
* **I94 Immigration**: The source data for I94 immigration data is available in local disk in the format of sas7bdat. This data comes from US National Tourism and Trade Office. The data dictionary is also included in this project for reference. The actual source of the data is from https://travel.trade.gov/research/reports/i94/historical/2016.html. This data is already uploaded to the workspace.

* **World Temperature Data**: This dataset came from Kaggle. This data is already uploaded to the workspace.

* **Airport Code**: This is a simple table with airport codes. The source of this data is from https://datahub.io/core/airport-codes#data. It is highly recommended to use it for educational purpose only but not for commercial or any other purpose. This data is already uploaded to the workspace.

* **U.S. City Demographic Data**: This data comes from OpenSoft link https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/

* Other validation text files such as *I94Addr.txt*, *I94CIT_I94RES.txt*, *I94Mode.txt*, *I94Port.txt** and **I94Visa.txt* extracted from the *I94_SAS_Labels_Descriptions.SAS*.

##### The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

#### Step 1: Scope the Project and Gather Data

##### Scope 

To make decision of project scope and the technical step solution we do data assessment on datasets:
* I94 Immigration
* World Temperature Data
* U.S. City Demographic Data
* Airport Code
* Other reference *I94_SAS_Labels_Descriptions.SAS*.

Tools will be used and import:
- Spark
- Python and its modules
- Pandas
- AWS Redshift cluster will be considered as an optional to run one or more ETL steps.

In [ ]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import re
import configparser
import os

Create config file *etl.cfg* for path parameters:
~~~
[DIR]
INPUT_DIR = .
OUTPUT_DIR = ./storage

[DATA]
I94_IMMI = ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
WORLD_TEMPE = ../../data2/GlobalLandTemperaturesByCity.csv
CITY_DEMOGRAPHIC = ./us-cities-demographics.csv
AIR_PORT = ./airport-codes_csv.csv

[SPLIT]
I94_IMMI_SPLITED_DIR = ./storage/.sas7bdat
WORLD_TEMPE_SPLITED_DIR = ./storage/.csv
~~~

Parse config file for path configurations - Done

In [ ]:
config = configparser.ConfigParser()
config.read('etl.cfg')

input_data_source = config.get('DIR','INPUT_DIR')
output_processed_data = config.get('DIR','OUTPUT_DIR')

i94immi_dataset = config.get('DATA','I94_IMMI')
worldtempe_dataset = config.get('DATA','WORLD_TEMPE')
citydemo_dataset = config.get('DATA','CITY_DEMOGRAPHIC')
airport_dataset = config.get('DATA','AIR_PORT')

Create Spark session:

In [ ]:
# Run on production only
spark = SparkSession.builder\
            .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
            .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .enableHiveSupport()\
            .getOrCreate()

In [ ]:
# Run on production only
local_spark = SparkSession.builder.appName("localSpark").getOrCreate()

##### Describe and Gather Data

Take a look on datasets description includes schema, sample records, number of rows, attributes, number of data file (if need).  And then choose datasets will be using for data modeling.

Dataset assessment by counting number of records and data size. Take a look on schema, data column structure, attributes and some of sample records.

- I94 Immigration

In [ ]:
# i94immi_dataset = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# i94_immi_df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

i94immi_dataset = 'immigration_data_sample.csv'
i94immi_df = pd.read_csv(i94immi_dataset,sep=",")

In [ ]:
# Show schema
i94immi_df.info()

In [ ]:
# Indexing
i94immi_df.index

In [ ]:
# Show dataset shape
i94immi_df.shape

In [ ]:
# Show dataset sample records
i94immi_df.head()

In [ ]:
# N/A values estimation
i94immi_df.isna().sum()

In [ ]:
# NULL values estimation
i94immi_df.isnull().sum()

- World Temperature Data

In [ ]:
# worldtempe_dataset = '../../data2/GlobalLandTemperaturesByCity.csv'
worldtempe_dataset = 'GlobalLandTemperaturesByCity_part9.csv'
worldtempe_df = pd.read_csv(worldtempe_dataset,sep=",")

In [ ]:
# Show schema
worldtempe_df.info()

In [ ]:
# Indexing
worldtempe_df.index

In [ ]:
# Show dataset shape
worldtempe_df.shape

In [ ]:
# Show dataset sample records
worldtempe_df.head()

In [ ]:
# N/A values estimation
worldtempe_df.isna().sum()

In [ ]:
# NULL values estimation
worldtempe_df.isnull().sum()

- U.S. City Demographic Data

In [ ]:
# citydemo_dataset = './us-cities-demographics.csv'
citydemo_dataset = './us-cities-demographics.csv'
citydemo_df = pd.read_csv(citydemo_dataset,sep=";")

In [ ]:
# Show schema
citydemo_df.info()

In [ ]:
# Indexing
citydemo_df.index

In [ ]:
# Show dataset shape
citydemo_df.shape

In [ ]:
# Show dataset sample records
citydemo_df.head()

In [ ]:
# N/A values estimation
citydemo_df.isna().sum()

In [ ]:
# NULL values estimation
citydemo_df.isnull().sum()

In [65]:
citydemo_df.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [66]:
# Check unique values of column "City"
citydemo_df['City'].nunique()

567

In [67]:
# Check unique values of column "State"
citydemo_df['State'].nunique()

49

In [68]:
citydemo_df.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

- Airport Code

In [ ]:
airport_dataset = './airport-codes_csv.csv'
airport_df = pd.read_csv(airport_dataset,sep=",")

In [ ]:
# Show schema
airport_df.info()

In [ ]:
# Indexing
airport_df.index

In [ ]:
# Show dataset shape
airport_df.shape

In [ ]:
# Show dataset sample records
airport_df.head()

In [ ]:
# N/A values estimation
airport_df.isna().sum()

In [ ]:
# NULL values estimation
airport_df.isnull().sum()

- Extract dictionary informations from *I94_SAS_Labels_Descriptions.SAS*.
    - I94CIT & I94RES --> i94cntyl.txt
    - I94PORT --> i94prtl.txt
    - I94MODE --> i94model.txt
    - I94ADDR --> i94addrl
    - I94VISA --> i94visa.txt

In [ ]:
with open('./I94_SAS_Labels_Descriptions.SAS') as f:
    f_content = f.read()
    f_content = f_content.replace('\t', '')

In [ ]:
def code_mapper(file, idx):
    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic

In [ ]:
i94_cit_and_res = code_mapper(f_content, "i94cntyl")
i94_port = code_mapper(f_content, "i94prtl")
i94_mode = code_mapper(f_content, "i94model")
i94_addr = code_mapper(f_content, "i94addrl")
i94_visa = {'1':'Business',
            '2': 'Pleasure',
            '3' : 'Student'}

In [ ]:
i94_cit_and_res

In [ ]:
i94_port

In [ ]:
i94_mode

In [ ]:
i94_addr

In [ ]:
i94_visa

In [ ]:
def convert_city_to_i94port(city):
    results = [v for k, v in i94_port.items() if re.match(city, k)]

In [ ]:
from pyspark.sql.functions import udf,col
convert_city_to_i94portUDF = udf(lambda z:convert_city_to_i94port(z))

temp_df_final = temp_df.withColumn('i94_port', convert_city_to_i94portUDF(col("city")))
temp_df_final.show(2)

Overview of analysis step targets:
- Staging datasets *Immigration Data*, *Temperature Data*, *U.S. City Demographic Data* and *Airport Code Table*.
- Create data modeling diagram as a star schema.
- Create and load data to fact table and dimension tables.
- Check data quality

#### Step 2: Explore and Assess the Data

##### Explore the Data

Dataset quality and validation issues recognition from gathering steps

- I94 Immigration
    - Add seperate columns of year, month and assign matching values of data.
    - Drop redundant columns. Keep necessary columns
    - Drop records with NA or NULL values
    - Drop duplicated records

- World Temperature Data
    - Convert data type of column 'dt' to datetime
    - Filter to keep US only
    - Lowercase column names and eliminate space
    - Drop records with NA or NULL values
    - Drop duplicated records
    - Cut out unnecessary columns

- U.S. City Demographic Data
    - Column name format should be changed to lowercase without space.
    - Population values should be change to percentage values.
    - Unique values of "City" and "State".
    - Drop redundant columns. Keep necessary columns
    - Drop records with NA or NULL values
    - Drop duplicated records

- Airport Code
    - Drop records with non-US values on column "Country".
    - Lowercase column name.
    - Drop records with NA or NULL values
    - Drop duplicated records

- Other reference *I94_SAS_Labels_Descriptions.SAS*.

##### Cleaning Steps

- Parse *I94_SAS_Labels_Descriptions.SAS*. for validations on staging steps

- Cleaning and staging I94 Immigration include columns:
    - i94cit = code for visitor origin country
    - i94port = code for destination USA city
    - arrdate = arrival date in the USA
    - i94mode = code for transportation mode
    - depdate = departure date from the USA
    - i94visa = code for visa type (reason for visiting)

- Cleaning and staging World Temperature Data include columns:
    - Date
    - AverageTemperature
    - City
    - State
    - Country

- Cleaning and staging U.S. City Demographic Data include columns:
    - City
    - State
    - Median Age
    - Male Population
    - Female Population
    - Total Population
    - Number of Veterans
    - Foreign-born
    - Average Household Size
    - Race
    - Count

- Cleaning and staging  Airport Code

#### Step 3: Define the Data Model

##### 3.1 Conceptual Data Model

Start schema diagram transformed
- Start_schema_diagram here

Fact table:
- Fact table fact_i94visits will contain informations from the i94 immigration data joined with daily average temperature on the port city and arrival date.

Dimension table:
- dim_us_ports contains informations like US port of entry code, city, state code and state name.
- dim_visa maps visa type which gives information like reason for visiting.
- dim_countries tells which country does visitor come from.
- dim_travelmode gives mode of transportation: air, land or sea.
- dim_demographicscontains median age and population informations about US port city.
- dim_date contains date information like year, month, day, week of year and weekday.

##### 3.2 Mapping Out Data Pipelines

The pipeline steps are described below:
- Load raw dataset from source into Spark dataframe: df_spark_i94, df_spark_dem and df_spark_temp for one month.
- Clean each Spark dataframe as decscibed in *Step 2 Cleaning steps* and write each cleaned dataframe into parquet as staging table: stage_i94_immigration, stage_cities_demographics and stage_uscities_temperatures.
- Create and load dimension tables: dim_us_ports, dim_visa, dim_countries, dim_travelmode and dim_demographics.
- Create and load fact table fact_i94_visits joining stage_i94_immigration and stage_uscities_temperatures.
- Create and load dimension tables and dim_date.

#### Step 4: Run Pipelines to Model the Data

##### 4.1 Create the data model

Build the data pipelines to create the data model.

##### 4.2 Data Quality Checks

Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:

* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
* Unit tests for the scripts to ensure they are doing the right thing
* Source/Count checks to ensure completeness

Run Quality Checks

##### 4.3 Data dictionary

The first dimension table will contain events from the I94 immigration data. The columns below will be extracted from the immigration dataframe:
* i94yr = 4 digit year
* i94mon = numeric month
* i94cit = 3 digit code of origin city
* i94port = 3 character code of destination city
* arrdate = arrival date
* i94mode = 1 digit travel code
* depdate = departure date
* i94visa = reason for immigration

The second dimension table will contain city temperature data. The columns below will be extracted from the temperature dataframe:
* i94port = 3 character code of destination city (mapped from immigration data during cleanup step)
* AverageTemperature = average temperature
* City = city name
* Country = country name
* Latitude= latitude
* Longitude = longitude

The fact table will contain information from the I94 immigration data joined with the city temperature data on i94port:
* i94yr = 4 digit year
* i94mon = numeric month
* i94cit = 3 digit code of origin city
* i94port = 3 character code of destination city
* arrdate = arrival date
* i94mode = 1 digit travel code
* depdate = departure date
* i94visa = reason for immigration
* AverageTemperature = average temperature of destination city

#### Step 5: Complete Project Write Up

- Clearly state the rationale for the choice of tools and technologies for the project.
    * Spark was chosen since it can easily handle multiple file formats (including SAS) containing large amounts of data. 
    * Spark SQL was chosen to process the large input files into dataframes and manipulated via standard SQL join operations to form additional tables.

- Propose how often the data should be updated and why.
    * The data should be updated monthly in conjunction with the current raw file format.
    * Case 2
    * Case 3

- Write a description of how you would approach the problem differently under the following scenarios:
    * The data was increased by 100x.
    * If the data was increased by 100x, we would no longer process the data as a single batch job. We could perhaps do incremental updates using a tool such as Uber's Hudi. We could also consider moving Spark to cluster mode using a cluster manager such as Yarn.

- The data populates a dashboard that must be updated on a daily basis by 7am every day.
    * If the data needs to populate a dashboard daily to meet an SLA then we could use a scheduling tool such as Airflow to run the ETL pipeline overnight.
    * Others solution ???

- The database needed to be accessed by 100+ people.
    * If the database needed to be accessed by 100+ people, we could consider publishing the parquet files to HDFS and giving read access to users that need it. 
    * If the users want to run SQL queries on the raw data, we could consider publishing to HDFS using a tool such as Impala.